In [ ]:
import torch
a = torch.tensor([[[0.1,0.2],[0.5,0.6],[0.5,0.6],[0.5,0.6]],[[0.3,0.4],[0.5,0.8],[0.2,0.3],[0.4,0.5],]])
print(a)
# print(a.sum(dim=0).sum(dim=1))

# b = torch.cumsum(a, dim=1)
# print(b)
# print(b[:,:-1])

# c = torch.cat([torch.ones_like(a[:, :1]), 1.0 - b[:,:-1]], dim=1)

cdf = torch.cumsum(a, dim=1)
print(cdf)
c = torch.cat(
    [torch.ones_like(a[:, :1, :]),
     1.0 - cdf[:, :-1, :]],
    dim=1
)

print(c)


In [ ]:
## unit test for torch.where and torch.zeros_like
import torch
a1 = torch.rand((3,4))
a2 = torch.rand((3,3))

time= a1>0.5
value = a2>0.7

3,4,3

event_bool = time.unsqueeze(2) & value.unsqueeze(1)

print(time)
print(value)
print(event_bool)

tensor([[False, False,  True, False],
        [False, False, False,  True],
        [ True,  True,  True, False]])
tensor([[False,  True, False],
        [False, False, False],
        [ True,  True,  True]])
tensor([[[False, False, False],
         [False, False, False],
         [False,  True, False],
         [False, False, False]],

        [[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True],
         [False, False, False]]])


In [ ]:
import numpy as np
from typing import Any, Dict, Iterator, List, Mapping, Optional, Sequence, Set, Tuple
import scipy.sparse

def sparse_to_dense(sparse_data):
    """Convert sparse data to dense tensor"""
    shape = (10,1000)
    sparse_dict = {k: v for k, v in sparse_data.items()}
    
    # Handle the case where we have 'value' field in sparse data
    if "value" in sparse_dict:
        # For value sparse data, we need to return both time and value
        time_sparse = scipy.sparse.csr_array(
            (sparse_dict["time"], sparse_dict["indices"], sparse_dict["indptr"]), 
            shape=shape
        )
        
        # Initialize value tensor with NaN (None equivalent) instead of 0
        # Values can range from -inf to +inf, so 0 is not appropriate default
        value_dense = torch.full((10, 1000), float('nan'), dtype=torch.float32)
        
        # Fill in actual values only where they exist
        for i in range(len(sparse_dict["indptr"]) - 1):
            start = sparse_dict["indptr"][i]
            end = sparse_dict["indptr"][i + 1]
            if end > start:
                indices = sparse_dict["indices"][start:end]
                values = sparse_dict["value"][start:end]
                value_dense[i, indices] = torch.tensor(values, dtype=torch.float32)
        
        return torch.from_numpy(time_sparse.toarray()), value_dense
    else:
        # Regular time sparse data
        sparse_matrix = scipy.sparse.csr_array(
            (sparse_dict["time"], sparse_dict["indices"], sparse_dict["indptr"]), 
            shape=shape
        )
        return torch.from_numpy(sparse_matrix.toarray())


time_sparse: Dict[str, List[float]] = {
    "time": [],
    "indices": [],
    "indptr": [0],
}
result = {
        "time": np.array(time_sparse["time"]),
        "indices": np.array(time_sparse["indices"]),
        "indptr": np.array(time_sparse["indptr"]),
    }

time_non_numerical = sparse_to_dense(result)
print(time_non_numerical)

## LAB

In [ ]:
import pathlib      
from tqdm import tqdm
import meds_reader
import femr.stat_utils
a = femr.stat_utils.ReservoirSampler(100000)
meds_reader_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"
database = meds_reader.SubjectDatabase(meds_reader_path)
lab_dict = {}
numerical_values= []
for subject_id in tqdm(database):
    for event in database[subject_id].events:
        if event.numeric_value is not None:
            if event.code not in lab_dict:
                lab_dict[event.code] = [1,femr.stat_utils.ReservoirSampler(100000)]
                lab_dict[event.code][1].add(float(event.numeric_value),1)
            else: 
                lab_dict[event.code][0] += 1
                lab_dict[event.code][1].add(float(event.numeric_value),1)
            # print(event.code,event.numeric_value,event.text_value)


In [ ]:
import femr.models.tokenizer
import pickle
import meds
tokenizer_path = "/user/zj2398/cache/motor_mimic_bin_20/tokenizer"
ontology_path = "/user/zj2398/cache/motor_mimic_bin_20/ontology.pkl"
with open(ontology_path, 'rb') as f:
    ontology = pickle.load(f)
tokenizer = femr.models.tokenizer.HierarchicalTokenizer.from_pretrained(tokenizer_path, ontology=ontology)
print(len(tokenizer.code_lookup))

# subject count = 364627
sorted_items = sorted(lab_dict.items(), key=lambda x: x[1][0],reverse=True)
# print(sorted_items)
# print(len(sorted_items))
in_dict = {}
out_dict = {}
for key,value in sorted_items:
    if key in tokenizer.code_lookup and value[0] > 3647:
        in_dict[key] = value
    else:
        out_dict[key] = value

print(len(in_dict))
print(in_dict)
print(len(out_dict))



In [ ]:
# frequency of each code in the database
frequency_list = list(in_dict.keys())
idx_dict = {code: i for i, code in enumerate(frequency_list)}
# print(idx_dict)

subject_counts = [0] * len(frequency_list)
for subject_id in tqdm(database):
    # track which codes appeared for this subject
    seen_codes = set()
    for event in database[subject_id].events:
        if event.code in in_dict and event.numeric_value is not None:
            seen_codes.add(event.code)
    
    # update subject-level counts
    for code in seen_codes:
        subject_counts[idx_dict[code]] += 1

print(len(subject_counts))
print([i/364627 for i in subject_counts])
print(sum([0.999>(i/364627)>0.001 for i in subject_counts]))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# summary statistics
s = pd.Series(count_list)
summary = pd.DataFrame({
    "count": [s.size],
    "unique": [s.nunique()],
    "min": [s.min()],
    "p1": [s.quantile(0.01)],
    "p5": [s.quantile(0.05)],
    "p25": [s.quantile(0.25)],
    "median": [s.median()],
    "p75": [s.quantile(0.75)],
    "p95": [s.quantile(0.95)],
    "p99": [s.quantile(0.99)],
    "max": [s.max()]
})
print(summary.to_string(index=False))

# log-spaced bins
b = np.logspace(np.log10(min(count_list)), np.log10(max(count_list)), 60)
plt.figure()
plt.hist(count_list, bins=b)
plt.xscale('log')
plt.xlabel("Value (log scale)")
plt.ylabel("Frequency")
plt.title("Histogram with log-spaced bins")
plt.tight_layout()
plt.show()

In [ ]:
x = np.sort(count_list)
y = np.arange(1, x.size + 1) / x.size
plt.figure()
plt.plot(x, y, drawstyle="steps-post")
plt.xlabel("Value")
plt.ylabel("ECDF")
plt.title("Empirical CDF")
plt.tight_layout()
plt.show()

### Tokenizer

In [ ]:
import femr.models.tokenizer
import pickle
tokenizer_path = "/user/zj2398/cache/motor_mimic/tokenizer"
ontology_path = "/user/zj2398/cache/motor_mimic/ontology.pkl"
with open(ontology_path, 'rb') as f:
    ontology = pickle.load(f)
tokenizer = femr.models.tokenizer.HierarchicalTokenizer.from_pretrained(tokenizer_path, ontology=ontology)
print(tokenizer.dictionary.keys())

In [ ]:
# import json
# store_path = "/user/zj2398/cache/motor/tokenizer"
# with open("dictionary.json", "w", encoding="utf-8") as f:
#     json.dump(tokenizer.dictionary, f)
# print(len(tokenizer.dictionary['vocab']))
print(tokenizer.numeric_values)
print(tokenizer.numeric_indices)

In [ ]:
import pickle
from femr.models.tokenizer.hierarchical_tokenizer import HierarchicalTokenizer
import pathlib
import meds_reader
meds_reader_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"
database = meds_reader.SubjectDatabase(meds_reader_path)

pretraining_data_path = pathlib.Path("/user/zj2398/cache/deephit_tpp_8k")
task_path = pretraining_data_path / "motor_task.pkl"
with open(task_path, 'rb') as f:
    motor_task = pickle.load(f)
task_info = motor_task.pretraining_task_info
# list_task_info = list(i[0] for i in task_info)
print(task_info)
print([a[1] for a in motor_task.pretraining_task_info])
# print(motor_task.task_to_index_map)
# print(motor_task.numerical_task_to_index_map)

# for i in motor_task.numerical_task_to_index_map.keys():
#     a = i.split("//")
#     if len(a)< 2:
#         print(i)
# code_list = [i.split("//")[1] for i in motor_task.numerical_task_to_index_map.keys()]
# print(len(code_list))
# print(code_list)
# for i in code_list:
#     try:
#         int(i)
#     except:
#         print(i)
# lab_test_list = [51265,50885,50912,50821,225170,220045,220210]

# for i in lab_test_list:
#     if i not in code_list:
#         print(i)
# print(code_list)
'''

Platelet Count → 51265 (complete blood count). 
PMC
Bilirubin, Total → 50885 (chemistry). 
MIMIC
PhysioNet
Creatinine (serum/plasma) → 50912 (chemistry). 
GitLab
PaO₂ (arterial blood gas “PO2”) → 50821 (blood gas).
'''

# ontology_path = pretraining_data_path / 'ontology.pkl'
# tokenizer_path = pretraining_data_path / 'tokenizer'
# with open(ontology_path, 'rb') as f:
#     ontology = pickle.load(f)
# tokenizer = HierarchicalTokenizer.from_pretrained(tokenizer_path, ontology=ontology)

In [ ]:
import sys
import meds_reader
meds_reader_path = "/user/zj2398/cache/mimic/meds_v0.6_reader"
database = meds_reader.SubjectDatabase(meds_reader_path)
count = 0
# type_list = []
# numerical_value_list = ['LAB','SUBJECT_FLUID_OUTPUT','INFUSION_START','SUBJECT_WEIGHT_AT_INFUSION','INFUSION_END']
numerical_code_list = []

for subject_id in database:
    subject = database[subject_id]
    for event in subject.events:
        if event.numeric_value is not None:
            try:
                float_value = float(event.numeric_value)
                numerical_code_list.append(event.code)
            except:
                continue
            # print(event.code,event.numeric_value,event.text_value)
        # if :
        #     event_type = event.code.split("//")[0]
        #     if event_type not in type_list:
        #         type_list.append(event_type)
        #         print(event_type)
        #     # if count > 100:
        #     #     sys.exit()
        #     # print(event.code,event.numeric_value,event.text_value)
        #     try:
        #         # print(event.numeric_value)
        #         float_value = float(event.numeric_value)
        #         # if float_value < 0:
        #         #     print(event.code,event.numeric_value,event.time,event.text_value)
        #         #     count += 1
        #     except:
        #         continue
                # print(f"error {event.code} {event.numeric_value} {event.text_value}")

print(len(numerical_code_list))
print(numerical_code_list)
# event is always not None except for GENDER


# for subject_id in database:
#     subject = database[subject_id]
#     for event in subject.events:
#         if event.time is None and not event.code.startswith("GENDER"):
#             print(event.code,event.numeric_value,event.text_value)
            # float_value = float(event.numeric_value)
            # if float_value < 0:
            #     print(event.code,event.numeric_value,event.time,event.text_value)
            #     count += 1


'''
LAB
SUBJECT_FLUID_OUTPUT
INFUSION_START
SUBJECT_WEIGHT_AT_INFUSION
INFUSION_END
'''



In [3]:
import numpy as np
a = np.array([[1,2],[3,4],[5,6]])
b = a.tolist()
print(b[1])

[3, 4]


In [6]:
# print(len(numerical_code_list))
# print(numerical_code_list)

dict1 = {"a":1,"b":2}
print(list(dict1.keys()))

['a', 'b']


In [ ]:
import femr.models.tokenizer
print(tokenizer.string_lookup['unit'])
print(len(tokenizer.string_lookup['unit'])) 
# ['route', 'frequency', 'text_value', 'language', 'race', 'priority', 'marital_status', 'insurance', 'unit', 'statusdescription', 'ordercategorydescription']


In [ ]:
# a = list(tokenizer.code_lookup.keys())
# # print(len(a))
# print(a)
print(tokenizer.code_lookup)

### Motor_task

In [ ]:
import pickle
task_path = '/user/zj2398/cache/motor/motor_task.pkl'
with open(task_path, 'rb') as f:
    motor_task = pickle.load(f)
print(motor_task.time_bins)
a = motor_task.pretraining_task_info

In [ ]:
print(a[0])

In [ ]:
# meds_reader_convert mimic_meds mimic_meds_reader --num_threads 32
import os
import pandas as pd
base_path = "/user/zj2398/cache/mimic/mimic-3.1-meds/patient_outcome_tasks/task"
for dir in os.listdir(base_path):
    dir_path = os.path.join(base_path, dir)
    if os.path.isdir(dir_path):
        parquet_file = os.path.join(dir_path, f"{dir}.parquet")
        if os.path.exists(parquet_file):
        #     print(f"The parquet file {parquet_file} exists")
        # else:
            # df = pd.concat([df_train, df_tuning, df_test])
            # df = df.sort_values(by=["subject_id", "prediction_time"])
            # df.to_parquet(parquet_file)
            # df_train.to_parquet(os.path.join(dir_path, f"{dir}.parquet"))
            # df_val.to_parquet(os.path.join(dir_path, f"{dir}.parquet"))
            # df_test.to_parquet(os.path.join(dir_path, f"{dir}.parquet"))
    # if os.path.isdir(os.path.join(base_path, dir)):
    # print(dir)

In [ ]:
import pandas as pd
dir_path = "/user/zj2398/cache/mimic/mimic-3.1-meds/patient_outcome_tasks/task/long_los/tuning.parquet"
df = pd.read_parquet(dir_path)
print(df)
df2 = df.to_dict(orient="records")
print(df2)

In [ ]:
print(df2[0:5])

In [ ]:
import pickle
task_path = "/user/zj2398/cache/motor_mimic/motor_task.pkl"
with open(task_path, 'rb') as f:
    motor_task = pickle.load(f)
# print(motor_task.pretraining_task_codes)
code_dict = {}
for i in motor_task.pretraining_task_codes:
    code_type = i.split("//")[0]
    if code_type not in code_dict:
        code_dict[code_type] = 1
    else:
        code_dict[code_type] += 1
print(len(motor_task.pretraining_task_codes))
for i in code_dict:
    print(i, code_dict[i])

# print(motor_task.time_bins)
# print(motor_task.final_layer_size)


In [ ]:
import torch
# a = torch.tensor([
#   # pred point 0
#   [[True, False, False],    # bin 0
#    [False, False, False]],  # bin 1

#   # pred point 1
#   [[False, False, False],
#    [False, False, True]]
# ])
# a = torch.tensor([[[0.4906, 0.0603, 0.3938, 0.1182],
#          [0.8817, 0.2784, 0.2033, 0.4675],
#          [0.0389, 0.5167, 0.3233, 0.8253]],

#         [[0.5043, 0.1975, 0.6027, 0.8749],
#          [0.4877, 0.4640, 0.9564, 0.4848],
#          [0.4997, 0.4962, 0.2766, 0.7736]]])
# a = torch.randint(0, 2, (2, 3, 4), dtype=torch.bool)
b= torch.tensor([[[ True, False, False,  True],
         [ True, False,  True,  True],
         [ True, False,  True,  True]],

        [[False,  True,  True,  True],
         [ True,  True, False, False],
         [False,  True, False,  True]]])
c = torch.any(b, dim=1)
print(c)

In [ ]:
import numpy as np
a = np.array([float("-inf"),1,1,1,1,1,1,1,1,1,float("inf")])

def keep_unique_bins(value_samples, num_bins):
    unique_bins = np.unique(value_samples)
    unique_bins_augmented = np.concatenate([unique_bins,np.array((num_bins+1-len(unique_bins))*[float("inf")])])
    mask = np.concatenate([np.ones(len(unique_bins)),np.zeros(num_bins+1-len(unique_bins))])
    return unique_bins_augmented, mask

unique_bins, mask = keep_unique_bins(a, 10)
print(unique_bins,unique_bins.dtype)
print(mask)